Провести сравнение RNN, LSTM, GRU на датасете отзывов (из предыдущих занятий/материалов)

In [1]:
from sklearn import model_selection, preprocessing, metrics
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
import re
import xgboost, textblob, string
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import keras
from keras.models import Sequential, Model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.objectives import categorical_crossentropy
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, MaxPooling1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking
from keras.callbacks import EarlyStopping
from keras import optimizers
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from string import punctuation
from gensim.models import Word2Vec
nltk.download("punkt")
from tqdm import tqdm
%matplotlib inline

import warnings 
warnings.filterwarnings("ignore", category=Warning)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
data = pd.read_excel('/content/drive/My Drive/отзывы за лето.xls')

In [3]:
data.head(10)

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14
5,5,Всё удобно норм 👍👍👍,2017-08-14
6,5,Очень удобное приложение.,2017-08-14
7,5,Все устраивает,2017-08-14
8,5,У меня работает все четко. В отличии от банком...,2017-08-14
9,5,Очень все хорошо👍,2017-08-14


In [4]:
exclude = set(punctuation)
stop_words = set(get_stop_words("ru"))
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = nltk.word_tokenize(str(txt))
    txt = [morpher.parse(word)[0].normal_form for word in txt if word not in stop_words]
    return " ".join(txt)

data['text'] = data['Content'].apply(preprocess_text)
data = data[data['Rating'] != 3]
data['target'] = data['Rating'] > 3
data['target'] = data['target'].astype(int)
data.head()

,Rating,Content,Date,text,target
0,5,It just works!,2017-08-14,it just works,1
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,целое удобноной приложениеиз минус хотеть боль...,1
2,5,Отлично все,2017-08-14,отлично,1
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,зависать 1 работа антивирус ранее пользоваться...,1
4,5,"Очень удобно, работает быстро.",2017-08-14,удобно работать быстро,1


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['target'], test_size=0.15,
                                                    random_state=42, stratify=data['target'])

In [6]:
text_corpus_train = X_train.values
text_corpus_test = X_test.values

In [7]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_test = pad_sequences(sequences_test, maxlen=training_length)

In [8]:
epochs = 15
batch_size = 1024

In [9]:
y_train = y_train.values
y_test = y_test.values

In [10]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=64,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=[keras.metrics.AUC()])

early_stopping=EarlyStopping(monitor='val_loss')  

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/15
15/15 [==============================] - 3s 176ms/step - loss: 0.5419 - auc: 0.5202 - val_loss: 0.4008 - val_auc: 0.8679
Epoch 2/15
15/15 [==============================] - 2s 160ms/step - loss: 0.3656 - auc: 0.7597 - val_loss: 0.3137 - val_auc: 0.8813
Epoch 3/15
15/15 [==============================] - 2s 163ms/step - loss: 0.3007 - auc: 0.8569 - val_loss: 0.2781 - val_auc: 0.9040
Epoch 4/15
15/15 [==============================] - 2s 163ms/step - loss: 0.2470 - auc: 0.9103 - val_loss: 0.2481 - val_auc: 0.9214
Epoch 5/15
15/15 [==============================] - 2s 156ms/step - loss: 0.1981 - auc: 0.9422 - val_loss: 0.2308 - val_auc: 0.9376
Epoch 6/15
15/15 [==============================] - 2s 157ms/step - loss: 0.1553 - auc: 0.9668 - val_loss: 0.2150 - val_auc: 0.9480
Epoch 7/15
15/15 [==============================] - 3s 179ms/step - loss: 0.1173 - auc: 0.9822 - val_loss: 0.2263 - val_auc: 0.9464


In [11]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

3/3 [==============================] - 0s 14ms/step - loss: 0.2401 - auc: 0.9378


Test score: 0.2401239424943924
Test accuracy: 0.9378198981285095


In [12]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=64,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(LSTM(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=[keras.metrics.AUC()])

early_stopping=EarlyStopping(monitor='val_loss')  

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/15
15/15 [==============================] - 7s 445ms/step - loss: 0.5663 - auc_1: 0.5760 - val_loss: 0.4147 - val_auc_1: 0.8677
Epoch 2/15
15/15 [==============================] - 6s 410ms/step - loss: 0.3675 - auc_1: 0.7679 - val_loss: 0.3157 - val_auc_1: 0.8962
Epoch 3/15
15/15 [==============================] - 6s 431ms/step - loss: 0.2901 - auc_1: 0.8876 - val_loss: 0.2647 - val_auc_1: 0.9110
Epoch 4/15
15/15 [==============================] - 7s 449ms/step - loss: 0.2367 - auc_1: 0.9269 - val_loss: 0.2380 - val_auc_1: 0.9280
Epoch 5/15
15/15 [==============================] - 6s 419ms/step - loss: 0.1908 - auc_1: 0.9542 - val_loss: 0.2157 - val_auc_1: 0.9436
Epoch 6/15
15/15 [==============================] - 7s 471ms/step - loss: 0.1537 - auc_1: 0.9705 - val_loss: 0.2061 - val_auc_1: 0.9498
Epoch 7/15
15/15 [==============================] - 7s 459ms/step - loss: 0.1255 - auc_1: 0.9802 - val_loss: 0.2093 - val_auc_1: 0.9500


In [13]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

3/3 [==============================] - 0s 38ms/step - loss: 0.2404 - auc_1: 0.9390


Test score: 0.24038635194301605
Test accuracy: 0.938985288143158


In [14]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=64,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(GRU(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=[keras.metrics.AUC()])

early_stopping=EarlyStopping(monitor='val_loss')  

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/15
15/15 [==============================] - 6s 405ms/step - loss: 0.6303 - auc_2: 0.5915 - val_loss: 0.5095 - val_auc_2: 0.8839
Epoch 2/15
15/15 [==============================] - 5s 365ms/step - loss: 0.3908 - auc_2: 0.7208 - val_loss: 0.3218 - val_auc_2: 0.9033
Epoch 3/15
15/15 [==============================] - 5s 360ms/step - loss: 0.2863 - auc_2: 0.9128 - val_loss: 0.2614 - val_auc_2: 0.9183
Epoch 4/15
15/15 [==============================] - 6s 369ms/step - loss: 0.2198 - auc_2: 0.9436 - val_loss: 0.2324 - val_auc_2: 0.9274
Epoch 5/15
15/15 [==============================] - 5s 357ms/step - loss: 0.1741 - auc_2: 0.9605 - val_loss: 0.2239 - val_auc_2: 0.9372
Epoch 6/15
15/15 [==============================] - 5s 366ms/step - loss: 0.1419 - auc_2: 0.9732 - val_loss: 0.2146 - val_auc_2: 0.9402
Epoch 7/15
15/15 [==============================] - 6s 410ms/step - loss: 0.1133 - auc_2: 0.9835 - val_loss: 0.2209 - val_auc_2: 0.9436


In [15]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

3/3 [==============================] - 0s 25ms/step - loss: 0.2617 - auc_2: 0.9335


Test score: 0.2617008686065674
Test accuracy: 0.9335076212882996


При увеличении батча, LSTM потихоньку начинает обходить, но судя по тесту разница у них небольшая